In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import euclidean_score
import pearson_score

In [2]:
df_movie = pd.read_csv('movie.csv')
df_rating = pd.read_csv('rating.csv')
df_tag = pd.read_csv('tag.csv')

In [4]:
print(df_movie.head())
print(df_rating.head())
print(df_tag.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40
   userId  movieId            tag            timestamp
0      18     4141    Mark Waters  2009-04-24 18:19:40
1      

In [10]:
unique_movie = df_movie['movieId'].unique()
movie_id_name = {}
for m in unique_movie:
    movie_id_name[m] = (df_movie.loc[df_movie['movieId'] == m])['title']

In [15]:
unique_tag_list = []
for t in df_movie['genres']:
    temp = t.split('|')
    for word in temp:
        if word not in unique_tag_list:
            unique_tag_list.append(word)

In [16]:
unique_tag = {}
for t in unique_tag_list:
    unique_tag[t] = 0

In [17]:
print(unique_tag)

{'Adventure': 0, 'Animation': 0, 'Children': 0, 'Comedy': 0, 'Fantasy': 0, 'Romance': 0, 'Drama': 0, 'Action': 0, 'Crime': 0, 'Thriller': 0, 'Horror': 0, 'Mystery': 0, 'Sci-Fi': 0, 'IMAX': 0, 'Documentary': 0, 'War': 0, 'Musical': 0, 'Western': 0, 'Film-Noir': 0, '(no genres listed)': 0}


In [ ]:
X = {}
for m in unique_movie:
    template = unique_tag.copy()
    for 